## Ghost Writer Detector



In [25]:
from lyricsgenius import Genius
import re 

with open('genius_token.txt', 'r') as file:
    token = file.read()

genius = Genius(token,
                timeout=15,
                sleep_time=2,
                retries=5)



artists = ['Drake','Souja Boy','Quentin Miller','Kendrick Lamar','J Cole','Mac Miller','Saba','A Tribe Called Quest']

song_titles = []
song_lyrics = []

for artist in artists:
    song = genius.search_artist(artist, max_songs=None)
    art_songs = []
    art_song_title = []
    for s in song.songs:
        art_songs.append(s.lyrics)
        art_song_title.append(s.title)
    song_titles.append(art_song_title)
    song_lyrics.append(art_songs)

Searching for songs by Drake...

Song 1: "God’s Plan"
Song 2: "In My Feelings"
Song 3: "Hotline Bling"
Song 4: "One Dance"
Song 5: "Hold On, We’re Going Home"
Song 6: "Know Yourself"
Song 7: "Back To Back"
Song 8: "Family Matters"
Song 9: "All Me"
Song 10: "From Time"
Song 11: "The Motto"
Song 12: "Pound Cake / Paris Morton Music 2"
Song 13: "0 to 100 / The Catch Up"
Song 14: "Fake Love"
Song 15: "Marvins Room"
Song 16: "Push Ups"
Song 17: "Started from the Bottom"
Song 18: "Jumpman"
Song 19: "Passionfruit"
Song 20: "HYFR (Hell Ya Fucking Right)"
Song 21: "Forever"
Song 22: "Nice For What"
Song 23: "Energy"
Song 24: "Child’s Play"
Song 25: "Too Good"
Song 26: "Jungle"
Song 27: "Furthest Thing"
Song 28: "Over My Dead Body"
Song 29: "Headlines"
Song 30: "Worst Behavior"
Song 31: "Summer Sixteen"
Song 32: "The Language"
Song 33: "Nonstop"
Song 34: "Girls Love Beyoncé"
Song 35: "Take Care"
Song 36: "How Bout Now"
Song 37: "THE HEART PART 6"
Song 38: "Shot for Me"
Song 39: "Too Much"
Song 4

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [24]:
song_titles

[['[Intro]\nAnd they wishin\' and wishin\'\nAnd wishin\' and wishin\', they wishin\' on me\nYeah\n\n[Verse 1]\nI been movin\' calm, don\'t start no trouble with me\nTryna keep it peaceful is a struggle for me\nDon\'t pull up at 6 a.m. to cuddle with me\nYou know how I like it when you lovin\' on me\nI don\'t wanna die for them to miss me\nYes, I see the things that they wishin\' on me\nHope I got some brothers that outlive me\nThey gon\' tell the story, shit was different with me\n\n[Pre-Chorus]\nGod\'s plan, God\'s plan\nI hold back, sometimes, I won\'t, yeah\nI feel good, sometimes, I don\'t (Ayy, don\'t)\nI finessed down Weston Road (Ayy, \'nessed)\nMight go down a G-O-D (Yeah, wait)\nI go hard on Southside G (Yeah, wait)\nI make sure that north-side eat\nAnd still\n\n[Chorus]\nBad things\nIt\'s a lot of bad things that they wishin\' and wishin\'\nAnd wishin\' and wishin\', they wishin\' on me\nBad things\nIt\'s a lot of bad things that they wishin\' and wishin\'\nAnd wishin\' and w

In [ ]:
from lyricsgenius import Genius
import pandas as pd
import time

# -----------------------------
# CONFIGURATION
# -----------------------------
with open("genius_token.txt", "r") as f:
    token = f.read().strip()

genius = Genius(token, timeout=15, sleep_time=2, retries=5)

# List of artists
artists = ["Mac Miller", "Kendrick Lamar", "Quentin Miller", "Drake", "J. Cole", "A Tribe Called Quest", "Kanye West", "Soulja Boy","Jay-Z", "Big L"]

# Optionally: limit to certain albums for each artist
# Leave list empty [] to pull all albums
target_albums = {
    "Mac Miller": ["Swimming", "Circles", "K.I.D.S.", "The Divine Feminine", "Faces"],
    "Kendrick Lamar": ["DAMN.", "good kid, m.A.A.d city", "To Pimp a Butterfly", "Mr. Morale & the Big Steppers", "Section.80"],
    "Quentin Miller": ["Essentials, Vol. 2", "Q.M.", "Hey! Thanks a Lot 3", "na fr.", "Falco", "X.X."],
    "Drake": ["So Far Gone", "Take Care", "Nothing Was the Same", "If You're Reading This It's Too Late", "More Life", "Scorpion", "Certified Lover Boy"],
    "J. Cole": ["The Warm Up", "Friday Night Lights", "Cole World: The Sideline Story", "2014 Forest Hills Drive", "4 Your Eyez Only", "KOD"],
    "A Tribe Called Quest": ["People’s Instinctive Travels and the Paths of Rhythm", "The Low End Theory", "Midnight Marauders", "The Anthology"],
    "Kanye West": ["The College Dropout", "Late Registration", "Graduation", "808s & Heartbreak", "My Beautiful Dark Twisted Fantasy", "The Life of Pablo"],
    "Soulja Boy": ["Pretty Boy Millionares", "Greatest Entertainer Alive", "Tell Em TV", "iSouljaBoyTellem"],
    "Jay-Z": ["Reasonable Doubt", "The Blueprint", "The Black Album", "The Blueprint 3", "Watch the Throne", "4:44"],
    "Big L": ["Now or Never", "The Danger Zone", "Return of the Devil's Son", "139 & Lenox", "Lifestylez Ov Da Poor & Dangerous"]
}

# -----------------------------
# FUNCTION DEFINITIONS
# -----------------------------

def get_artist_id(artist_name):
    """Search Genius API for an artist's ID."""
    try:
        search = genius.search_artists(artist_name)
        return search["sections"][0]["hits"][0]["result"]["id"]
    except Exception as e:
        print(f"⚠️ Could not find artist ID for {artist_name}: {e}")
        return None


def get_albums_by_artist(artist_id):
    """Return all albums for a given artist ID."""
    all_albums = []
    page = 1
    while True:
        response = genius.artist_albums(artist_id, page=page)
        albums_page = response.get("albums", [])
        if not albums_page:
            break
        all_albums.extend(albums_page)
        page += 1
    return [
        {"album_name": a["name"], "album_id": a["id"], "url": a["url"]}
        for a in all_albums
    ]


def get_album_tracks(album_id):
    """Return all songs from a given album."""
    tracks = []
    page = 1
    while True:
        response = genius._make_request(
            f"albums/{album_id}/tracks",
            params_={"page": page},
            public_api=True
        )
        page_tracks = response.get("tracks", [])
        if not page_tracks:
            break
        tracks.extend(page_tracks)
        page += 1
    return [
        {"song_id": t["song"]["id"], "song_title": t["song"]["title"], "date": t["song"]["release_date_for_display"], "url": t["song"]["url"]}
        for t in tracks
    ]


def get_lyrics(url):
    """Fetch lyrics safely."""
    try:
        return genius.lyrics(song_url=url)
    except Exception as e:
        print(f"⚠️ Error fetching lyrics from {url}: {e}")
        return None


# -----------------------------
# MAIN LOOP
# -----------------------------

all_results = []
seen_tracks = set()

for artist_name in artists:
    print(f"\n🎤 Processing artist: {artist_name}")
    artist_id = get_artist_id(artist_name)
    if artist_id is None:
        continue

    albums = get_albums_by_artist(artist_id)
    if not albums:
        print(f"  ⚠️ No albums found for {artist_name}")
        continue

    df_albums = pd.DataFrame(albums)

    # Filter if target albums specified
    if target_albums.get(artist_name):
        df_albums = df_albums[df_albums["album_name"].isin(target_albums[artist_name])]

    for _, album_row in df_albums.iterrows():
        album_name = album_row["album_name"]
        album_id = album_row["album_id"]
        print(f"  💿 Getting tracks for album: {album_name}")

        tracks = get_album_tracks(album_id)
        for t in tracks:
            songid = t['song_id']
            if songid not in seen_tracks:
                seen_tracks.add(songid)
                print(f"     🎶 {t['song_title']}")
                lyrics = get_lyrics(t["url"])
                all_results.append({
                    "artist": artist_name,
                    "album": album_name,
                    "song_title": t["song_title"],
                    "date": t['date'],
                    "lyrics": lyrics
                })
                time.sleep(1)  # be nice to the API
        newest_50 = genius.artist_songs(artist_id, sort='release_date', per_page=50)
        print(f"  🚀 Getting latest songs from: {artist_name}")
        for t in newest_50['songs']:
            songid = t['id']
            if songid not in seen_tracks:
                seen_tracks.add(songid)
                s = genius.song(songid)
                if s['song']['album'] is not None:
                    album_name = s['song']['album']['name']
                else: 
                    album_name = None
                print(f"     🎶 {t['title']}")
                lyrics = get_lyrics(t["url"])
                all_results.append({
                    "artist": artist_name,
                    "album": album_name,
                    "song_title": t["title"],
                    "date": t["release_date_for_display"],
                    "lyrics": lyrics
                })
                time.sleep(1)
        popular_50 = genius.artist_songs(artist_id, sort='popularity', per_page=50)
        print(f"  🔥 Getting most popular tracks from: {artist_name}")
        for t in newest_50['songs']:
            songid = t['id']
            if songid not in seen_tracks:
                seen_tracks.add(songid)
                s = genius.song(songid)
                if s['song']['album'] is not None:
                    album_name = s['song']['album']['name']
                else: 
                    album_name = None
                print(f"     🎶 {t['song_title']}")
                lyrics = get_lyrics(t["url"])
                all_results.append({
                    "artist": artist_name,
                    "album": album_name,
                    "song_title": t["title"],
                    "date": t["release_date_for_display"],
                    "lyrics": lyrics
                })
                time.sleep(1)

# -----------------------------
# SAVE RESULTS
# -----------------------------
df = pd.DataFrame(all_results)
df.head()
print("\n✅ Done! Saved to 'selected_album_lyrics.csv'")



🎤 Processing artist: Mac Miller


In [ ]:
get_artist_id("Drake")
albums = get_albums_by_artist(130)


In [95]:
df = pd.DataFrame(albums)
df[df['album_name'] == "If You’re Reading This It’s Too Late"]

,album_name,album_id,url
17,If You’re Reading This It’s Too Late,119674,https://genius.com/albums/Drake/If-youre-readi...


In [97]:
get_album_tracks(119674)

[{'song_id': 703734,
  'song_title': 'Legend',
  'url': 'https://genius.com/Drake-legend-lyrics'},
 {'song_id': 703741,
  'song_title': 'Energy',
  'url': 'https://genius.com/Drake-energy-lyrics'},
 {'song_id': 703738,
  'song_title': '10 Bands',
  'url': 'https://genius.com/Drake-10-bands-lyrics'},
 {'song_id': 703654,
  'song_title': 'Know Yourself',
  'url': 'https://genius.com/Drake-know-yourself-lyrics'},
 {'song_id': 703742,
  'song_title': 'No Tellin’',
  'url': 'https://genius.com/Drake-no-tellin-lyrics'},
 {'song_id': 703728,
  'song_title': 'Madonna',
  'url': 'https://genius.com/Drake-madonna-lyrics'},
 {'song_id': 500858,
  'song_title': '6 God',
  'url': 'https://genius.com/Drake-6-god-lyrics'},
 {'song_id': 703743,
  'song_title': 'Star67',
  'url': 'https://genius.com/Drake-star67-lyrics'},
 {'song_id': 703749,
  'song_title': 'Preach',
  'url': 'https://genius.com/Drake-preach-lyrics'},
 {'song_id': 703746,
  'song_title': 'Wednesday Night Interlude',
  'url': 'https://

In [98]:
test = genius.song(703654)

In [104]:
test['song']['release_date']

'2015-02-13'

In [122]:
get_artist_id('Mac Miller')
mac = genius.artist_songs(820, sort="release_date", per_page = 50)


In [182]:
for i in range(len(mac['songs'])):
    if mac['songs'][i]['title'] == "5 Dollar Pony Rides":
        print(i)
        
t = genius.song(mac['songs'][19]['id'])

18


In [184]:
t['song']['album'] is None

True

In [174]:
tt = genius.song(mac['songs'][0]['id'])

In [ ]:
tt['song']['album']['name']

'Harlem’s Finest: Return of the King'